In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, PolynomialFeatures, scale
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression, RFE
import datetime
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer

import warnings # supress warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('600K US Housing Properties.csv', low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   property_url        600000 non-null  object 
 1   property_id         600000 non-null  int64  
 2   address             600000 non-null  object 
 3   street_name         599869 non-null  object 
 4   apartment           14815 non-null   object 
 5   city                599999 non-null  object 
 6   state               599999 non-null  object 
 7   latitude            529122 non-null  float64
 8   longitude           529122 non-null  float64
 9   postcode            599970 non-null  object 
 10  price               600000 non-null  float64
 11  bedroom_number      443845 non-null  float64
 12  bathroom_number     471733 non-null  float64
 13  price_per_unit      435365 non-null  float64
 14  living_space        447847 non-null  float64
 15  land_space          515119 non-nul

In [4]:
description = df.describe(include='all')
description

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
count,600000,6.000000e+05,600000,599869,14815,599999,599999,529122.000000,529122.000000,599970,...,600000,600000,0.0,0.0,600000.0,600000,444524,0.0,0.0,600000.000000
unique,600000,NaN,598588,339224,2664,7977,25,NaN,NaN,10820,...,7,2,NaN,NaN,NaN,1,34372,NaN,NaN,NaN
top,https://www.zillow.com/homedetails/3-Plat-83-1...,NaN,"(undisclosed Address), Rockport, TX 78382",(undisclosed Address),1,Chicago,TX,NaN,NaN,84043,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,NaN,2022-04-24 07:34:15,Coldwell Banker Realty,NaN,NaN,NaN
freq,1,NaN,36,1713,312,14138,146636,NaN,NaN,1102,...,354366,383365,NaN,NaN,NaN,600000,5936,NaN,NaN,NaN
mean,NaN,8.885042e+08,NaN,NaN,NaN,NaN,NaN,36.282379,-105.813906,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000498
std,NaN,9.724708e+08,NaN,NaN,NaN,NaN,NaN,5.673355,13.464633,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.022318
min,NaN,2.700000e+01,NaN,NaN,NaN,NaN,NaN,18.985142,-165.408250,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
25%,NaN,5.402143e+07,NaN,NaN,NaN,NaN,NaN,32.612112,-117.346079,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
50%,NaN,2.066090e+08,NaN,NaN,NaN,NaN,NaN,35.403568,-101.897378,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
75%,NaN,2.066867e+09,NaN,NaN,NaN,NaN,NaN,39.661674,-95.354245,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000


In [5]:
df.columns

Index(['property_url', 'property_id', 'address', 'street_name', 'apartment',
       'city', 'state', 'latitude', 'longitude', 'postcode', 'price',
       'bedroom_number', 'bathroom_number', 'price_per_unit', 'living_space',
       'land_space', 'land_space_unit', 'broker_id', 'property_type',
       'property_status', 'year_build', 'total_num_units', 'listing_age',
       'RunDate', 'agency_name', 'agent_name', 'agent_phone',
       'is_owned_by_zillow'],
      dtype='object')

In [6]:
#Selected just the Texas data to reduce column number so model can fit and predict
df = df[df.state == 'TX']

In [7]:
#Land space is an important metric for the price of the house but this dataset has multiple units
#All the values with 'acres' unit changed to 'sqft'
df.loc[df['land_space_unit']=='acres', 'land_space'] = df['land_space']*43560.00
df.loc[df['land_space_unit']=='acres', 'land_space_unit']='sqft'
df[['land_space','land_space_unit']]

,land_space,land_space_unit
180271,6969.600,sqft
180273,12632.400,sqft
180275,11325.600,sqft
180277,15246.000,sqft
180279,6000.000,sqft
...,...,...
446346,8145.720,sqft
446348,3920.400,sqft
446350,4356.000,sqft
446352,4299.372,sqft


## First Data Cleaning Method contains dropping some rows and features that is not related to price and filling the Na values with the median value.

In [8]:
df1 = df
df1 = df1.drop(df[df.living_space == 0].index)
df1 = df1.drop(df[df.price == 0].index)
df1 = df1.drop(columns=['property_url', 'property_id', 'address', 'street_name', 'apartment', 'city', 'state', 'price_per_unit', 'land_space_unit', 'broker_id','property_status', 'year_build', 'total_num_units', 'listing_age',
       'RunDate', 'agency_name', 'agent_name', 'agent_phone',
       'is_owned_by_zillow'])

In [9]:
df1.isna().sum()

latitude           19530
longitude          19530
postcode               0
price                  0
bedroom_number     33135
bathroom_number    19597
living_space       34065
land_space         22698
property_type          0
dtype: int64

In [10]:
#filling the rest with median values
df1['bedroom_number'] = df1['bedroom_number'].fillna(df1['bedroom_number'].median())
df1['bathroom_number'] = df1['bathroom_number'].fillna(df1['bathroom_number'].median())
df1['living_space'] = df1['living_space'].fillna(df1['living_space'].median())
df1['land_space'] = df1['land_space'].fillna(df1['land_space'].median())
df1 = df1.dropna(subset=['latitude', 'longitude'])

In [11]:
list = ["bedroom_number", "bathroom_number", "living_space", "land_space", "price"]

In [12]:
for i in list:
    q_low = df[i].quantile(0.01)
    q_hi  = df[i].quantile(0.99)
    df1 = df1[(df[i] < q_hi) & (df1[i] > q_low)]

## Second Data Cleaning Method contains dropping some rows and features but filling the Na values with the zero and expanding dataframe doubling with is_na columns and dummy variables.

In [13]:
df2=df
df2_isna=df2.isna()
df2_isna

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
180271,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180273,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180275,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180277,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180279,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
446348,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
446350,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
446352,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False


In [14]:
df2_isna.columns=[c+'_isna' for c in df2_isna.columns]
df2_isna

,property_url_isna,property_id_isna,address_isna,street_name_isna,apartment_isna,city_isna,state_isna,latitude_isna,longitude_isna,postcode_isna,...,property_type_isna,property_status_isna,year_build_isna,total_num_units_isna,listing_age_isna,RunDate_isna,agency_name_isna,agent_name_isna,agent_phone_isna,is_owned_by_zillow_isna
180271,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180273,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180275,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180277,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
180279,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
446348,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
446350,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
446352,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False


In [15]:
df2=pd.concat([df2.fillna(0), df2_isna.astype(float)], axis=1)
df2

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type_isna,property_status_isna,year_build_isna,total_num_units_isna,listing_age_isna,RunDate_isna,agency_name_isna,agent_name_isna,agent_phone_isna,is_owned_by_zillow_isna
180271,https://www.zillow.com/homedetails/1116-Saint-...,27399441,"1116 Saint Johns Dr, El Paso, TX 79903",Saint Johns Dr,0,El Paso,TX,31.786737,-106.428020,79903,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
180273,https://www.zillow.com/homedetails/1101-Apache...,27498318,"1101 Apache St, El Paso, TX 79925",Apache St,0,El Paso,TX,31.784021,-106.404450,79925,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
180275,https://www.zillow.com/homedetails/452-Val-Ver...,27497333,"452 Val Verde St, El Paso, TX 79905",Val Verde St,0,El Paso,TX,31.761911,-106.432330,79905,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
180277,https://www.zillow.com/homedetails/4600-Cumber...,27416021,"4600 Cumberland Cir, El Paso, TX 79903",Cumberland Cir,0,El Paso,TX,31.790165,-106.435960,79903,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
180279,https://www.zillow.com/homedetails/4524-Durazn...,27432630,"4524 Durazno Ave, El Paso, TX 79905",Durazno Ave,0,El Paso,TX,31.775480,-106.438230,79905,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446346,https://www.zillow.com/homedetails/6409-Tyler-...,53071725,"6409 Tyler Ct, Plano, TX 75023",Tyler Ct,0,Plano,TX,33.059647,-96.710945,75023,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
446348,https://www.zillow.com/homedetails/770-Pierre-...,26647700,"770 Pierre Ln, Plano, TX 75023",Pierre Ln,0,Plano,TX,33.044098,-96.707720,75023,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
446350,https://www.zillow.com/homedetails/1116-Canoe-...,53085667,"1116 Canoe Ln, Plano, TX 75023",Canoe Ln,0,Plano,TX,33.069946,-96.712920,75023,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
446352,https://www.zillow.com/homedetails/6925-Sugar-...,53085648,"6925 Sugar Maple Crk, Plano, TX 75023",Sugar Maple Crk,0,Plano,TX,33.070004,-96.713900,75023,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [16]:
# A crucial categorical feature 'property_type' is categorical variable so I wanted to transform that to a numeric one
dummies=pd.get_dummies(df2[['postcode','property_type']], drop_first=True)

In [17]:
df2 = pd.concat([df2,dummies],axis='columns').drop(columns=['postcode','property_type'])

In [18]:
list = ["bedroom_number", "bathroom_number", "living_space", "land_space", "price"]

In [19]:
for i in list:
    q_low = df2[i].quantile(0.01)
    q_hi  = df2[i].quantile(0.99)
    df2 = df2[(df2[i] < q_hi) & (df2[i] > q_low)]

In [20]:
description = df2.describe(include='all')

## Third Data Cleaning Method doesn't contain dropping some rows and features and fills the Na values with the zero and expanding dataframe doubling with is_na columns and dummy variables.

In [21]:
df3=df
df3_isna=df3.isna()
df3_isna.columns=[c+'_isna' for c in df3_isna.columns]
df3=pd.concat([df3.fillna(0), df3_isna.astype(float)], axis=1)
dummies=pd.get_dummies(df3[['postcode','property_type']], drop_first=True)
df3 = pd.concat([df3,dummies],axis='columns').drop(columns=['postcode','property_type'])

In [22]:
list = ["bedroom_number", "bathroom_number", "living_space", "land_space", "price"]
for i in list:
    q_low = df3[i].quantile(0.01)
    q_hi  = df3[i].quantile(0.99)
    df3 = df3[(df3[i] < q_hi) & (df3[i] > q_low)]

In [23]:
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgbm

## Splitting, scaling and fitting the model with first Method's dataframe

In [24]:
features=df1.describe().columns.drop(['price'])
target=['price']
X=df1[features]
y=df1[target]
    
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42)
len(X_train),len(X_test),len(y_train),len(y_test)
    
numeric=['latitude', 'longitude', 'bedroom_number', 'bathroom_number', 'living_space', 'land_space']
sc=StandardScaler()
X_train[numeric]=sc.fit_transform(X_train[numeric])
X_test[numeric]=sc.transform(X_test[numeric])
    
#generic function to fit model and return metrics for every algorithm
def boost_models(x):
    #transforming target variable through quantile transformer
    regr_trans = TransformedTargetRegressor(regressor=x, transformer=QuantileTransformer(output_distribution='normal'))
    regr_trans.fit(X_train, y_train)
    yhat = regr_trans.predict(X_test)
    train_pred = regr_trans.predict(X_train)
    algoname= x.__class__.__name__
    return algoname, round(r2_score(y_test, yhat),3), round(mean_absolute_error(y_test, yhat),2), round(metrics.mean_squared_error(y_test, yhat, squared=False),2), round(metrics.mean_squared_error(y_train, train_pred, squared=False),2)

algo=[LinearRegression(), Ridge(), Lasso(), RandomForestRegressor(), DecisionTreeRegressor(), GradientBoostingRegressor(), lgbm.LGBMRegressor()]
score=[]
for a in algo:
    score.append(boost_models(a))

#Collate all scores in a table
Model1 = pd.DataFrame(score, columns=['Model', 'R2 Score', 'MAE', 'Test RMSE', 'Train RMSE'])

## Splitting, scaling and fitting the model with second Method's dataframe

In [25]:
features=df2.describe().columns.drop(['price'])
target=['price']
X=df2[features]
y=df2[target]
    
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42)
len(X_train),len(X_test),len(y_train),len(y_test)
    
numeric=['latitude', 'longitude', 'bedroom_number', 'bathroom_number', 'living_space', 'land_space']
sc=StandardScaler()
X_train[numeric]=sc.fit_transform(X_train[numeric])
X_test[numeric]=sc.transform(X_test[numeric])
    
#generic function to fit model and return metrics for every algorithm
def boost_models(x):
    #transforming target variable through quantile transformer
    regr_trans = TransformedTargetRegressor(regressor=x, transformer=QuantileTransformer(output_distribution='normal'))
    regr_trans.fit(X_train, y_train)
    yhat = regr_trans.predict(X_test)
    train_pred = regr_trans.predict(X_train)
    algoname= x.__class__.__name__
    return algoname, round(r2_score(y_test, yhat),3), round(mean_absolute_error(y_test, yhat),2), round(metrics.mean_squared_error(y_test, yhat, squared=False),2), round(metrics.mean_squared_error(y_train, train_pred, squared=False),2)

algo=[LinearRegression(), Ridge(), Lasso(), RandomForestRegressor(), DecisionTreeRegressor(), GradientBoostingRegressor(), lgbm.LGBMRegressor()]
score=[]
for a in algo:
    score.append(boost_models(a))

#Collate all scores in a table
Model2 = pd.DataFrame(score, columns=['Model', 'R2 Score', 'MAE', 'Test RMSE', 'Train RMSE'])

## Splitting, scaling and fitting the model with third Method's dataframe

In [26]:
features=df3.describe().columns.drop(['price'])
target=['price']
X=df3[features]
y=df3[target]
    
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42)
len(X_train),len(X_test),len(y_train),len(y_test)
    
numeric=['latitude', 'longitude', 'bedroom_number', 'bathroom_number', 'living_space', 'land_space']
sc=StandardScaler()
X_train[numeric]=sc.fit_transform(X_train[numeric])
X_test[numeric]=sc.transform(X_test[numeric])
    
#generic function to fit model and return metrics for every algorithm
def boost_models(x):
    #transforming target variable through quantile transformer
    regr_trans = TransformedTargetRegressor(regressor=x, transformer=QuantileTransformer(output_distribution='normal'))
    regr_trans.fit(X_train, y_train)
    yhat = regr_trans.predict(X_test)
    train_pred = regr_trans.predict(X_train)
    algoname= x.__class__.__name__
    return algoname, round(r2_score(y_test, yhat),3), round(mean_absolute_error(y_test, yhat),2), round(metrics.mean_squared_error(y_test, yhat, squared=False),2), round(metrics.mean_squared_error(y_train, train_pred, squared=False),2)

algo=[LinearRegression(), Ridge(), Lasso(), RandomForestRegressor(), DecisionTreeRegressor(), GradientBoostingRegressor(), lgbm.LGBMRegressor()]
score=[]
for a in algo:
    score.append(boost_models(a))

#Collate all scores in a table
Model3 = pd.DataFrame(score, columns=['Model', 'R2 Score', 'MAE', 'Test RMSE', 'Train RMSE'])

In [27]:
#list dataframe you want to append
frame = [Model1, Model2, Model3]

#new dataframe to store append result
myDataFrame = pd.DataFrame()

myDataFrame = pd.concat(frame, keys=["1st Method", "2nd Method", "3rd Method"])
    
myDataFrame

Model  R2 Score        MAE  Test RMSE  \
1st Method 0           LinearRegression     0.363  145157.38  267632.05   
           1                      Ridge     0.363  145156.64  267629.87   
           2                      Lasso    -0.068  204574.73  346526.57   
           3      RandomForestRegressor     0.709   86913.44  181047.34   
           4      DecisionTreeRegressor     0.473  121493.33  243378.23   
           5  GradientBoostingRegressor     0.607  107527.61  210256.50   
           6              LGBMRegressor     0.697   93348.89  184577.41   
2nd Method 0           LinearRegression     0.942   30045.90   49431.52   
           1                      Ridge     0.942   30046.85   49413.00   
           2                      Lasso     0.450  107510.75  151714.49   
           3      RandomForestRegressor     0.999    1300.33    4723.14   
           4      DecisionTreeRegressor     0.998    3063.99    8572.65   
           5  GradientBoostingRegressor     0.998    6016.63    9908.83   
           6              LGBMRegressor     0.999    3723.60    7822.42   
3rd Method 0           LinearRegression     0.942   30045.90   49431.52   
           1                      Ridge     0.942   30046.85   49413.00   
           2                      Lasso     0.450  107510.75  151714.49   
           3      RandomForestRegressor     0.999    1275.67    4720.07   
           4      DecisionTreeRegressor     0.998    3049.11    8728.56   
           5  GradientBoostingRegressor     0.998    6016.98    9909.13   
           6              LGBMRegressor     0.999    3723.60    7822.42   

              Train RMSE  
1st Method 0   264324.27  
           1   264322.28  
           2   348864.72  
           3    77939.42  
           4     1923.78  
           5   203235.94  
           6   171888.14  
2nd Method 0    48241.73  
           1    48536.19  
           2   151628.97  
           3     1532.00  
           4       14.99  
           5     9286.04  
           6     6703.68  
3rd Method 0    48241.73  
           1    48536.19  
           2   151628.97  
           3     1514.45  
           4       14.99  
           5     9286.04  
           6     6703.68